# Rigid Body Movement Analysis

This notebook is to analyze the rigid body movement of M2 with the configuration file: **ts_mtm2/config/sysconfig
/Configurable_File_Description_20180831T092556_surrogate_handling.csv**.
The target is to understand:

1. Change of the net force and moment in the movement.
2. Force balance system is working or not.

It is noted that the correctness of force balance system would need a more detailed analysis in the future.

## Summary

1. The position data of hardpoints matches well with the independent measurement system (IMS).
2. The force balance system should be working under small movement.
3. The calculation of y position looks a little weird in the middle of movement (see the test of z rotation angle from 0 urad to -45 urad).

## Import Modules

This notebook needs to setup the **ts_m2com** and **ts_aos_utilsts** under the **notebooks/.user_setups**, which depends on the **ts_tcpip**.
You also need to have **ts_mtm2** under the **WORK/** directory to read the confiugration files to do the analysis.

In [ ]:
%matplotlib inline
%matplotlib widget
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from astropy.time import Time

from lsst.ts.m2com import MockControlClosedLoop

from lsst.ts.aos.utils.utility_m2 import (
    retrieve_efd_client,
    get_data_position,
    get_data_net_force,
    get_data_force_balance,
    get_data_net_moment,
    get_data_net_displacement,
    get_data_force,
    get_xy_actuators,
    draw_forces,
    plot_positions,
    plot_net_force,
    plot_net_moment,
    plot_force_balance,
    plot_net_displacement,
    plot_force_error,
)

from lsst_efd_client import EfdClient

## Declaration of User-defined Functions

In [ ]:
async def show_all_data(
    control_closed_loop: MockControlClosedLoop,
    efd_client: EfdClient,
    time_start: Time,
    time_end: Time,
    position: list,
) -> None:
    """
    Show all the data.

    Parameters
    ----------
    control_closed_loop : `lsst.ts.m2.com.MockControlClosedLoop`
        Mock control closed loop instance.
    efd_client : `lsst_efd_client.efd_helper.EfdClient`
        EFD client.
    time_start : `astropy.time.core.Time`
        Start time.
    time_end : `astropy.time.core.Time`
        End time.
    position : `list`
        Position of the rigid body moment in (x, y, z, rx, ry, rz) in the unit of m and rad.
    """

    # Calculate the theoretical displacement (note the final unit is um)
    displacements_theory = MockControlClosedLoop.rigid_body_to_actuator_displacement(
        control_closed_loop.get_actuator_location_axial(),
        control_closed_loop.get_actuator_location_tangent(),
        control_closed_loop.get_radius(),
        position[0],
        position[1],
        position[2],
        position[3],
        position[4],
        position[5],
    ) * 1e6

    # Get the x, y position of actuators
    xy_actuators = get_xy_actuators(control_closed_loop)

    # Query data
    data_position, time_operation_position = await get_data_position(efd_client, "position", time_start, time_end)
    data_ims, time_operation_ims = await get_data_position(efd_client, "positionIMS", time_start, time_end)

    data_force, time_operation_force = await get_data_net_force(efd_client, time_start, time_end)
    data_moment, time_operation_moment = await get_data_net_moment(efd_client, time_start, time_end)

    data_force_balance, time_operation_force_balance = await get_data_force_balance(efd_client, time_start, time_end)

    displacements_experiment = await get_data_net_displacement(efd_client, time_start, time_end)

    data_collected_axial, data_collected_tangent = await get_data_force(efd_client, time_start, time_end)

    # Draw the diagrams
    plot_positions(data_position, time_operation_position, data_ims, time_operation_ims)

    plot_net_force(data_force, time_operation_force)
    plot_net_moment(data_moment, time_operation_moment)    

    plot_force_balance(data_force_balance, time_operation_force_balance)

    plot_net_displacement(displacements_theory, displacements_experiment)

    draw_forces(
        xy_actuators,
        data_collected_axial["measured"][-1]-data_collected_axial["measured"][0],
        data_collected_tangent["measured"][-1]-data_collected_tangent["measured"][0],
    )
    
    plot_force_error(data_collected_axial, data_collected_tangent)

## Instantiate the MockControlClosedLoop

In [ ]:
config_path = Path.home() / "WORK" / "ts_mtm2" / "config" / "parameter_files"
filepath_cell_geom = Path.home() / "WORK" / "ts_mtm2" / "config" / "cellGeom.json"

In [ ]:
control_closed_loop = MockControlClosedLoop()
control_closed_loop.load_file_cell_geometry(filepath_cell_geom)

## Instantiate the EFD

Notice that the UTC time is used when doing the query.

In [ ]:
efd_client = retrieve_efd_client(is_summit=True)

## Move x Position from 0 mm to 0.5 mm

In [ ]:
time_start = Time("2023-05-16T18:41:30", scale="utc", format="isot")
time_end = Time("2023-05-16T18:43:30", scale="utc", format="isot")

position = [0.5 * 1e-3, 0, 0, 0, 0, 0]
await show_all_data(control_closed_loop, efd_client, time_start, time_end, position)

## Move x Position from 0 mm to -0.5 mm

In [ ]:
time_start = Time("2023-05-16T18:50:30", scale="utc", format="isot")
time_end = Time("2023-05-16T18:52:50", scale="utc", format="isot")

position = [-0.5 * 1e-3, 0, 0, 0, 0, 0]
await show_all_data(control_closed_loop, efd_client, time_start, time_end, position)

## Move y Position from 0 mm to 0.5 mm

In [ ]:
time_start = Time("2023-05-16T18:58:30", scale="utc", format="isot")
time_end = Time("2023-05-16T19:01:00", scale="utc", format="isot")

position = [0, 0.5 * 1e-3, 0, 0, 0, 0]
await show_all_data(control_closed_loop, efd_client, time_start, time_end, position)

## Move y Position from 0 mm to -0.5 mm

In [ ]:
time_start = Time("2023-05-16T19:10:20", scale="utc", format="isot")
time_end = Time("2023-05-16T19:12:40", scale="utc", format="isot")

position = [0, -0.5 * 1e-3, 0, 0, 0, 0]
await show_all_data(control_closed_loop, efd_client, time_start, time_end, position)

## Move z Position from 0 mm to 0.5 mm

In [ ]:
time_start = Time("2023-05-16T19:17:10", scale="utc", format="isot")
time_end = Time("2023-05-16T19:18:00", scale="utc", format="isot")

position = [0, 0, 0.5 * 1e-3, 0, 0, 0]
await show_all_data(control_closed_loop, efd_client, time_start, time_end, position)

## Move z Position from 0 mm to -0.5 mm

In [ ]:
time_start = Time("2023-05-16T19:23:30", scale="utc", format="isot")
time_end = Time("2023-05-16T19:24:20", scale="utc", format="isot")

position = [0, 0, -0.5 * 1e-3, 0, 0, 0]
await show_all_data(control_closed_loop, efd_client, time_start, time_end, position)

## Move x Rotation from 0 urad to 45 urad (= 9.28 arcsec)

In [ ]:
time_start = Time("2023-05-16T19:40:40", scale="utc", format="isot")
time_end = Time("2023-05-16T19:41:00", scale="utc", format="isot")

position = [0, 0, 0, 45 * 1e-6, 0, 0]
await show_all_data(control_closed_loop, efd_client, time_start, time_end, position)

## Move x Rotation from 0 urad to -45 urad (= -9.28 arcsec)

In [ ]:
time_start = Time("2023-05-16T19:43:10", scale="utc", format="isot")
time_end = Time("2023-05-16T19:43:40", scale="utc", format="isot")

position = [0, 0, 0, -45 * 1e-6, 0, 0]
await show_all_data(control_closed_loop, efd_client, time_start, time_end, position)

## Move y Rotation from 0 urad to 45 urad (= 9.28 arcsec)

In [ ]:
time_start = Time("2023-05-16T19:45:00", scale="utc", format="isot")
time_end = Time("2023-05-16T19:45:10", scale="utc", format="isot")

position = [0, 0, 0, 0, 45 * 1e-6, 0]
await show_all_data(control_closed_loop, efd_client, time_start, time_end, position)

## Move y Rotation from 0 urad to -45 urad (= -9.28 arcsec)

In [ ]:
time_start = Time("2023-05-16T19:46:30", scale="utc", format="isot")
time_end = Time("2023-05-16T19:47:00", scale="utc", format="isot")

position = [0, 0, 0, 0, -45 * 1e-6, 0]
await show_all_data(control_closed_loop, efd_client, time_start, time_end, position)

## Move z Rotation from 0 urad to 45 urad (= 9.28 arcsec)

In [ ]:
time_start = Time("2023-05-16T19:34:20", scale="utc", format="isot")
time_end = Time("2023-05-16T19:35:00", scale="utc", format="isot")

position = [0, 0, 0, 0, 0, 45 * 1e-6]
await show_all_data(control_closed_loop, efd_client, time_start, time_end, position)

## Move z Rotation from 0 urad to -45 urad (= -9.28 arcsec)

In [ ]:
time_start = Time("2023-05-16T19:37:20", scale="utc", format="isot")
time_end = Time("2023-05-16T19:38:00", scale="utc", format="isot")

position = [0, 0, 0, 0, 0, -45 * 1e-6]
await show_all_data(control_closed_loop, efd_client, time_start, time_end, position)

## Move x, y Positions from 0 mm to 0.35 mm

In [ ]:
time_start = Time("2023-05-16T19:52:30", scale="utc", format="isot")
time_end = Time("2023-05-16T19:54:40", scale="utc", format="isot")

position = [0.35 * 1e-3, 0.35 * 1e-3, 0, 0, 0, 0]
await show_all_data(control_closed_loop, efd_client, time_start, time_end, position)

## Move x, y Positions from 0 mm to -0.35 mm

In [ ]:
time_start = Time("2023-05-16T19:58:50", scale="utc", format="isot")
time_end = Time("2023-05-16T20:01:00", scale="utc", format="isot")

position = [-0.35 * 1e-3, -0.35 * 1e-3, 0, 0, 0, 0]
await show_all_data(control_closed_loop, efd_client, time_start, time_end, position)

In [ ]:
plt.close("all")